### (2,3,4) curves

We will use the $(2,3,4)$ factorization curves to create a Wigner kernel. According to Sainz these curves can't be used to generate MUBs using Kanat's method, but it seems to me that they can.

In [1]:
%display latex

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [2, 2]

In [2]:
N = 3
d = 2**N
F = GF(d, 'x')
x = F.gen()

In [3]:
def Proj(u, v=None):
    # Expects vectors
    if not v:
        v = u
    u = matrix(u).transpose()
    v = matrix(v).transpose()
    return u.tensor_product(v.conjugate_transpose())

Id = identity_matrix(SR, 2**N)

def toInt(k):
    return list(F).index(k)

def chi(k):
    return exp(2 * pi * I * int(k.trace()) / 2)

In [4]:
R = PolynomialRing(Integers(4), 't')
t = R.gen()

In [5]:
poly = t^3 + 2*t^2 + t - 1
factor(poly)

t^3 + 2*t^2 + t + 3

In [7]:
GR = R.quotient(poly, 'w')
w = GR.gen()
GR

Univariate Quotient Polynomial Ring in w over Ring of integers modulo 4 with modulus t^3 + 2*t^2 + t + 3

In [8]:
T = [GR(0)] + [w**j for j in range(1, 2**N - 1)] + [GR(1)]

In [234]:
def TeichLift(k):
    return T[toInt(k)]

hat = TeichLift

In [12]:
def Fourier():
    s = zero_matrix(SR, 2**N, 2**N)
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            s[i,j] = chi(a * b) / sqrt(2^N)
    return s
FF = Fourier()

def Z(a):
    return diagonal_matrix([chi(a * k) for k in F])

def X(b):
    return FF.conjugate_transpose() * Z(b) * FF

The chosen curves aren't parametrized by a single parameter and so it would seem the Galois ring method would not be applicable. In reality all that is needed that the subspaces generated the curves are totally isotropic. The curves are
$$
\begin{align*}
    f_0(\alpha) &= 0, \\
    f_x(\alpha) &= x^6 \alpha + x^3 \alpha^2 + x^5 \alpha^4, \\
    f_{x^2}(\alpha) &= x^2 \alpha + x^5 \alpha^2 + x^6 \alpha^4, \\
    f_{x^3}(\alpha) &= x^4 \alpha + x^3 \alpha^2 + x^5 \alpha^4, \\
    f_{x^4}(\alpha) &= x^3 \alpha, \\
    f_{x^5}(\alpha) &= x^5 \alpha + x^5 \alpha^2 + x^6 \alpha^4, \\
    f_{x^6}(\alpha) &= x \alpha + x^2 \alpha^2 + x \alpha^4, \\
    f_{1}(\alpha) &= \alpha + x^2 \alpha^2 + x \alpha^4.
\end{align*}
$$
Where $x$ is a generator of the field. The immediate question is how to order the curves. I have kept the order in which Andres sent them to me, and so I simply label each curve by an element of the field according to its index.

In [16]:
curves = [
    lambda a: 0,
    lambda a: x^6 * a + x^3 * a^2 + x^5 * a^4,
    lambda a: x^2 * a + x^5 * a^2 + x^6 * a^4,
    lambda a: x^4 * a + x^3 * a^2 + x^5 * a^4,
    lambda a: x^3 * a,
    lambda a: x^5 * a + x^5 * a^2 + x^6 * a^4,
    lambda a: x * a + x^2 * a^2 + x * a^4,
    lambda a: a + x^2 * a^2 + x * a^4
]

In [17]:
# We have to lift the curves to the Teichmüller set, 
# thankfully ^x = w
curves_T = [
    lambda a: 0,
    lambda a: w^6 * a + w^3 * a^2 + w^5 * a^4,
    lambda a: w^2 * a + w^5 * a^2 + w^6 * a^4,
    lambda a: w^4 * a + w^3 * a^2 + w^5 * a^4,
    lambda a: w^3 * a,
    lambda a: w^5 * a + w^5 * a^2 + w^6 * a^4,
    lambda a: w * a + w^2 * a^2 + w * a^4,
    lambda a: a + w^2 * a^2 + w * a^4
]

Let's see if the subspaces generated by the curves are totally isotropic.

In [18]:
for j, curve in enumerate(curves):
    subspace = [(a, curve(a)) for a in F]
    # The curves are subspaces
    for p1 in subspace:
        for p2 in subspace:
            if (p1[0] + p2[0], p1[1] + p2[1]) not in subspace:
                raise Exception('Curve is not a subspace.', j)
    
    # The curves are not abelian
    for p1 in subspace:
        for p2 in subspace:
            c = p1[0] * p2[1] - p1[1] * p2[0]
            if c.trace() != 0:
                raise Exception('Not isotropic!', j)

In her proof, the only thing required of the $f$ is that for all elements of the field
$$
\chi(\xi f(\alpha)) = \chi(\alpha f(\xi)),
$$
where $\chi$ is the additive group character $\chi(\kappa) = \omega^{tr(\kappa)}$ where $\omega$ is a primitive root of unity. This equality is true if and only if
$$
tr(\xi f(\alpha)) = tr(\alpha f(\xi)).
$$
This is true for symplectic presemifields, that is to say, presemifield operations that generate a symplectic spread. In our case the curves aren't generated by a single parameter and so can't really be seen as a binary operation and hence as a presemifield. Even so, we can test that each curve satisfies this property:

In [44]:
for c in curves:
    for alpha in F:
        for xi in F:
            if (alpha * c(xi)).trace() != (xi * c(alpha)).trace():
                raise Exception('Property does not hold for curve {}'.format(c))

The same property should hold for the Teichmüller set according to Kanat.

In [45]:
for c in curves_T:
    for alpha in T:
        for xi in T:
            if (alpha * c(xi)).trace() != (xi * c(alpha)).trace():
                raise Exception('Property does not hold for curve {}'.format(c))

Following Sainz's proof there doesn't seem to be a reason why these curves can't be used to construct the appropriate rotation operators. But before moving on let's check to see if the curves generate an affine plain, at least in the sense that the curves only intersect at the origin.

In [73]:
subspaces = [[(k, f(k)) for k in F] for f in curves]
for i, s1 in enumerate(subspaces):
    for j, s2 in enumerate(subspaces):
        if i != j:
            cap = list(set(s1) & set(s2))
            if cap != [(F(0), F(0))]:
                raise Exception('Non trivial intersection!', i, j, cap)

All is well, this means that at a single point, there will only be $2^N+1$ curves that pass through it:

In [78]:
for alpha in F:
    for beta in F:
        cnt = 0
        for f in curves:
            for nu in F:
                if beta == f(alpha) + nu:
                    cnt += 1
        if cnt != 8:
            raise Exception('More curves passed through the point ({}, {})!'.format(alpha, beta))

In [285]:
# x is the generator.
# The lifting operator ^ : F -> T is a multiplicative group
# homomorphism so ^(x * k) = ^x * ^k.

def phi(tau, nu):
    return (I)^(int((tau * nu).trace()))

def D(a, b):
    return phi(hat(a), hat(b)) * Z(a) * X(b)

def V(mu):
    s = zero_matrix(SR, 2^N, 2^N)
    curve = curves_T[toInt(mu)]
    for i, k in enumerate(F):
        a = TeichLift(k)
        s += phi(a, curve(a)) * Proj(vector(FF[:,i]))
    return s

In [230]:
for k in F:
    if V(x)[:,toInt(k)] != (V(x) * X(k) * Id[:,0]):
        raise Exception('Ordering is not right!')

In [36]:
# Orthonormality
def isOrthonormal(m):
    return m.conjugate_transpose() * m == Id

# MUBs
def isMUB(m1, m2):
    m = (
        m1.conjugate_transpose() * m2
    ).apply_map(lambda t: abs(t)^2)
    return m == ones_matrix(2^N, 2^N) / (2^N)

def checkMUBs(mubs):
    if type(mubs) != list:
        mubs_list = []
        for i in range(2^N+1):
            mubs_list.append(mubs[(i*2^N):(i+1)*2^N,:])
    else:
        mubs_list = mubs
            
    for i in range(2^N+1):
        for j in range(2^N+1):
            if i == j:
                if not isOrthonormal(mubs_list[i]):
                    raise Exception(
                        'Encountered a basis that is not orthonormal!',
                        i
                    )
            else:
                if not isMUB(mubs_list[i], mubs_list[j]):
                    raise Exception(
                        'Encountered non-MUB pairs of basis!',
                        i, j
                    )
    return True

In [37]:
mubs = [FF] + [V(mu) for mu in F]
checkMUBs(mubs)

True

We can see that we do indeed obtain valid MUBs. Now let's review Sainz's proof that we obtain delta functions when the kernel constructed from one set of MUBs is used to obtain the Wigner function of a state coming from some other set of MUBs. The central step is the evaluation of the following inner product:
$$
\langle \psi^\mu_\nu | \psi^{f_x}_\kappa \rangle,
$$
where $|\psi^\mu_\nu\rangle$ is the eigenvector associated to the straight line parametrized by $\mu$ and that passes the origin at $\nu$, while $|\psi^{f_x}_\kappa\rangle$ is the state associated to the curve $f_x$ that passes the origin at $\kappa$.

First a quick check that 
$$
\sum_{\kappa} \chi(\xi \kappa) = 2^N \delta_{\xi,0}.
$$

In [50]:
[sum([chi(xi * k) for k in F]) for xi in F]

[8, 0, 0, 0, 0, 0, 0, 0]

An important step in the calculation of the inner product is the use the of the rotation operator coefficients recurence properties, in particular:
$$
c_{\eta, f_x(\eta)} c_{\eta', f_x(\eta')}
= c_{\eta-\eta', f_x(\eta) - f_x(\eta')} \chi\left(-\eta' f_x(\eta - \eta')\right).
$$
Let's see if this holds for the Galois ring induced phase:
$$
c_{\eta, f_x(\eta)} = \Phi(\eta, f_x(\eta)) = \omega^{tr(\hat{\eta} f_x(\hat{\eta})}.
$$

In [59]:
hat = TeichLift # for notation
for j, f in enumerate(curves):
    for eta in F:
        for etap in F:
            f_hat = curves_T[j]
            l = phi(hat(eta), f_hat(hat(eta))) * phi(hat(etap), f_hat(hat(etap))).conjugate()
            r = phi(hat(eta + etap), f_hat(hat(eta + etap))) * chi(etap * f(eta + etap))
            if l != r:
                raise Exception('Recurrence property is not satisfied!', j, eta, etap)

So the recurrence relation is satisfied appropriately and the curves satisfy the isotropic property which means Sainz's proof should be applicable. In order to computationally verify the proof we need to calculate the standard set of MUBs.

In [79]:
def VW(mu):
    s = zero_matrix(SR, 2^N, 2^N)
    for i, k in enumerate(F):
        s += phi(TeichLift(k), TeichLift(mu * k)) * Proj(vector(FF[:,i]))
    return s

In [81]:
mubs306 = [FF] + [VW(mu) for mu in F]
checkMUBs(mubs306)

True

For a given point $(\alpha,\beta)$ we need to compute which lines are incident to it, and then obtain the associated states $|\psi_\nu^\mu\rangle$, where $\mu$ and $\nu$ parametrize the line.

In [173]:
def getStateFromMu(a, b, mu, M=mubs306):
    # non-Fourier states
    nu = b - a * mu
    return M[toInt(mu) + 1][:, toInt(nu)]

In [174]:
def getStateFromCurve(curve_param, nu, M=mubs):
    # non-Fourier states
    return M[toInt(curve_param) + 1][:, toInt(nu)]

According the calculation we have:
$$
|\langle \psi_\nu^\mu | \psi_\kappa^{f_x}\rangle|^2
= \frac{1}{d} \sum_{\xi} c_{\xi, f_x(\xi)} c_{\xi, \xi\mu}^* \chi(\xi(\kappa + \beta + \mu\alpha))
\delta_{\mu\xi, f_x(\xi)}.
$$

Let's verify this.

In [175]:
def inp(u, v):
    u = vector(u)
    v = vector(v)
    return u.conjugate() * v

In [179]:
hat = TeichLift
def Sainz(a, b, mu, curve_param, k):
    s = 0
    for xi in F:
        curve_idx = toInt(curve_param)
        if mu * xi == curves[curve_idx](xi):
            c_mu = phi(hat(xi), hat(xi * mu)).conjugate()
            c_fx = phi(hat(xi), curves_T[curve_idx](hat(xi)))
            char = chi(xi * (k + b + mu * a))
            s += c_mu * c_fx * char / 2^N
    return s

In [195]:
# this takes a while, but it will verify this part
# of the proof.

for alpha in F:
    for beta in F:
        for curve_param in F:
            for kappa in F:
                for mu in F:
                    u = getStateFromMu(alpha, beta, mu)
                    v = getStateFromCurve(curve_param, kappa)
                    
                    if abs(inp(u, v))**2 != Sainz(alpha, beta, mu, curve_param, kappa):
                        raise Exception(
                            'Inner products do not match!',
                            alpha, beta, mu, curve_param, kappa
                        )

All is well with that part! Now onto the final important part of the proof. Summing over $\mu \in F$ should give us:
$$
\sum_\mu
|\langle \psi_\nu^\mu | \psi_\kappa^{f_x}\rangle|^2
= 1 + \frac{1}{d} \sum_{\xi \in F} \chi(\xi(\kappa + \beta + f_x(\alpha))) - \frac{1}{d}
= 1 - \frac{1}{d} + \delta_{\kappa, \beta + f_x(\alpha)}.
$$

In [222]:
alpha = x + 1
beta  = x
curve_param = x^2
kappa = x

v = getStateFromCurve(curve_param, kappa)

l = 0
for mu in F:
    u = getStateFromMu(alpha, beta, mu)
    l += abs(inp(u, v))^2

r = 7/8 + int(kappa == beta - curves[toInt(curve_param)](alpha))

l, r

(3/8, 7/8)

And it's not working...

I can construct the Wigner kernel just as well. And in fact it satisfies all of the Stratonovich-Weyl properties.

In [256]:
def Wootters(a, b):
    op = Proj(vector(FF[:, toInt(a)]))
    for xi in F:
        nu = b - curves[toInt(xi)](a)
        v = V(xi)[:, toInt(nu)]
        op += Proj(vector(v))
    return op - Id

In [257]:
for a in F:
    for b in F:
        op = Wootters(a,b)
        if op != op.conjugate_transpose():
            raise Exception('Not self-adjoint.')

In [258]:
for a in F:
    for b in F:
        if Wootters(a, b).trace() != 1:
            raise Exception('Not unit trace.')

In [250]:
for a in F:
    for b in F:
        op1 = Wootters(a, b)
        for k in F:
            for l in F:
                op2 = Wootters(k, l)
                if (op1 * op2.conjugate_transpose()).trace() != 2^N * int(a == k) * int(b == l):
                    raise Exception('Not trace orthogonal.')

In [288]:
k = x
l = 1
for a in F:
    for b in F:
        if D(k,l) * Wootters(a, b) * D(k,l).conjugate_transpose() != Wootters(a+k, b+l):
            raise Exception('Not covariant.')

In [259]:
def WignerMatrix(state):
    op = zero_matrix(SR, 2^N, 2^N)
    for i, a in enumerate(F):
        for j, b in enumerate(F):
            op[i, j] = (state * Wootters(a, b)).trace() / 2^N
    return op

In [280]:
wf = WignerMatrix(Proj(vector(mubs306[6][:,0])))
wf

[-1/32  1/32  1/32  1/32 -1/32 -1/32  1/32  3/32]
[ 1/32 -1/32  3/32 -1/32  1/32  1/32 -1/32  1/32]
[ 3/32  1/32  1/32  1/32 -1/32 -1/32  1/32 -1/32]
[ 1/32 -1/32 -1/32  3/32  1/32  1/32 -1/32  1/32]
[ 1/32 -1/32 -1/32 -1/32  1/32  1/32  3/32  1/32]
[ 1/32  3/32 -1/32 -1/32  1/32  1/32 -1/32  1/32]
[-1/32  1/32  1/32  1/32 -1/32  3/32  1/32 -1/32]
[-1/32  1/32  1/32  1/32  3/32 -1/32  1/32 -1/32]